In [1]:
import os
import sys

from django.utils import timezone

import requests
import json
from datetime import datetime

sys.path.append('/home/abezpalov/daven.io/daven/')
os.environ['DJANGO_SETTINGS_MODULE'] = 'daven.settings'

from django.core.wsgi import get_wsgi_application
application = get_wsgi_application()

from dataminer.models import *

In [2]:
# Функция загрузки открытых данных
def load(method, pair = None, limit = None, timeout = 10):

    if method in ('depth', 'trades'):
        limit = 2000

    if pair and limit:
        url = 'https://api.liqui.io/api/3/{}/{}?limit={}'.format(method, pair, limit)
    elif pair:
        url = 'https://api.liqui.io/api/3/{}/{}'.format(method, pair)
    else:
        url = 'https://api.liqui.io/api/3/{}'.format(method)

    r = requests.get(url, timeout = timeout)

    return json.loads(r.text)

In [3]:
# get_pairs
method = 'info'

pairs = load(method)['pairs']

pairs

{'1st_btc': {'decimal_places': 8,
  'fee': 0.25,
  'hidden': 1,
  'max_amount': 1000000000.0,
  'max_price': 2.0,
  'min_amount': 1e-08,
  'min_price': 1e-08,
  'min_total': 0.0001},
 '1st_eth': {'decimal_places': 8,
  'fee': 0.25,
  'hidden': 1,
  'max_amount': 1000000.0,
  'max_price': 2.0,
  'min_amount': 1e-08,
  'min_price': 1e-07,
  'min_total': 0.001},
 '1st_usdt': {'decimal_places': 8,
  'fee': 0.25,
  'hidden': 1,
  'max_amount': 1000000.0,
  'max_price': 200.0,
  'min_amount': 0.01,
  'min_price': 0.001,
  'min_total': 1.0},
 'adx_btc': {'decimal_places': 8,
  'fee': 0.25,
  'hidden': 0,
  'max_amount': 1000000.0,
  'max_price': 2.0,
  'min_amount': 1e-08,
  'min_price': 1e-08,
  'min_total': 0.0001},
 'adx_eth': {'decimal_places': 8,
  'fee': 0.25,
  'hidden': 0,
  'max_amount': 1000000.0,
  'max_price': 40.0,
  'min_amount': 1e-08,
  'min_price': 1e-07,
  'min_total': 0.001},
 'adx_usdt': {'decimal_places': 8,
  'fee': 0.25,
  'hidden': 0,
  'max_amount': 1000000.0,
  'max_

In [4]:
currencies = set()

for key in pairs:
    for currency_ in key.split('_'):
        currencies.add(currency_)

currencies

{'1st',
 'adx',
 'ae',
 'ans',
 'ant',
 'ast',
 'bat',
 'bcap',
 'bcc',
 'bmc',
 'bnt',
 'btc',
 'cfi',
 'cvc',
 'dash',
 'dct',
 'dgd',
 'dnt',
 'edg',
 'eng',
 'eos',
 'eth',
 'gbg',
 'gno',
 'gnt',
 'golos',
 'gup',
 'hmq',
 'icn',
 'incnt',
 'ind',
 'knc',
 'ltc',
 'lun',
 'mana',
 'mco',
 'mgo',
 'mln',
 'myst',
 'net',
 'neu',
 'oax',
 'omg',
 'pay',
 'plu',
 'pro',
 'ptoy',
 'qrl',
 'qtum',
 'rep',
 'req',
 'rlc',
 'round',
 'salt',
 'san',
 'sbd',
 'sngls',
 'snm',
 'snt',
 'steem',
 'storj',
 'stx',
 'taas',
 'time',
 'tkn',
 'tnt',
 'trst',
 'trx',
 'usdt',
 'ven',
 'vsl',
 'waves',
 'wings',
 'xid',
 'xzc',
 'zrx'}

In [ ]:
# По каждой паре получаем текущую информацию, ордеры, историю торгов
for pair in pairs:
    method = 'ticker'
    data = load(method, pair)
    save(data, method, pair)